## Κατηγοροποίηση πελατών
* Δηλώνω τις βιβλιοθήκες που θα χρησιμοποίησω

In [2]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

* fact_table.txt -> το αρχείο έχει εξαχθεί από το data warehouse και αντιστοιχεί στον πίνακα fact Table όπως προέκυψε μετά τη διαδικασία ETL

In [3]:
data = pd.read_csv('fact_table.txt', sep=",")

* Διώχνω όποιο βιβλίο έχει αγοραστεί λιγότερο από 50 φορές

In [4]:
data_filtered =data.groupby('Book').filter(lambda x : len(x)>50)

* Διώχνω τα βιβλία με μηδενική αξιολόγηση καθώς αυτό σημαίνει πως στην πραγματικότητα δεν αξιολογήθηκαν από τους χρήστες καθώς η βαθμολογία κυμαίνεται από 1 εώς 10

In [5]:
data_filtered= data_filtered[data_filtered['Rating_Score'] != 0]

* Αφαιρώ κάποιες στήλες

In [6]:
data_final = data_filtered.drop([ 'Publisher', 'Book_Author', 'Reader'], axis=1)

* Υπολογίζω το μέσο όρο αξιολογήσεων κάθε βιβλίου

In [7]:
mean = data_final.groupby('Book')['Rating_Score'].mean().reset_index()

* Κάνω την ίδια διαδικασία για το mean

In [8]:
data_std = pd.read_csv('fact_table.txt', sep=",")

* Αφαιρώ γραμμές με μηδενικές αξιολογήσεις

In [9]:
data_std= data_std[data_std['Rating_Score'] != 0]

* Αφαιρώ όσα βιβλία έχουν λιγότερες από 50 αξιολογήσεις

In [10]:
data_std_filtered =data_std.groupby('Book').filter(lambda x : len(x)>50)

* Διαγράφω κάποιες στήλες

In [11]:
data_std_final = data_std_filtered.drop(['Publisher', 'Book_Author', 'Reader'], axis=1)

* Υπολογίζω την τυπική απόκλιση ανά βιβλίο

In [12]:
std = data_std_final.groupby('Book')['Rating_Score'].std().reset_index()

* Συνδέω το μέσο όρο που υπολογίστηκε προηγουμένως με την τυπική απόκλιση

In [13]:
finale = pd.merge(mean, std, on='Book')

* ευρέως αρεστά: πάνω από 6 μέσο όρο και std < 1.670443859682947  (1)
* αμφιλογόμενα: std > 1.670443859682947                           (2)
* ευρέως μη αρεστά: κάτω από έξι με std<1.670443859682947         (3)

In [14]:
def f(row):
    val = 2
    if row['Rating_Score_x'] >= 6 and row['Rating_Score_y'] < 1.670443859682947:
        val = 1
    if row['Rating_Score_y'] > 1.670443859682947:
        val = 2
    if row['Rating_Score_y'] < 1.670443859682947 and row['Rating_Score_x'] <6:
        val = 3
    return val

* Αποδίδω σε κάθε γραμμή μία ετικέτα

In [15]:
finale['label'] = finale.apply(f, axis=1)

In [17]:
finale